In [1]:
from sqlalchemy import create_engine
import pandas.io.sql as sqlio
import os
from dotenv import load_dotenv

In [2]:
# Curry function to initialize postgres engine and return read_query function

def make_db_url(protocol, user, password, db, host, port):
    return f'{protocol}://{user}:{password}@{host}:{port}/{db}'

def get_read_query(protocol, user, password, host, port, db):
    db_url = make_db_url(protocol, user, password, db, host, port)
    engine = create_engine(db_url)
    
    def read_query(query, verbose=True):
        if verbose:
            print(query, '\n')
            
        with engine.connect() as conn:
            df = sqlio.read_sql_query(query, conn)

        return df

    return read_query

In [3]:
# Template functions generic sql queries

## Base query
def base_query(table, select='*', where=None, order_by=None, limit=None):
    query = f"SELECT {select}"
    query += f" FROM {table}"
    query += f" WHERE {where}" if where else ""
    query += f" ORDER BY {order_by}" if order_by else ""
    query += f" LIMIT {limit}" if limit else ""
    return query

## Aggregate
def aggregate_query(table, col_agg_dict, where=None, order_by=None, limit=None):
    last_iter = sum([len(aggs) for aggs in col_agg_dict.values()])
    curr_iter = 1
    select = ""
    for col, aggs in col_agg_dict.items(): 
        for agg in aggs:
            if agg == "COUNT":
                select += "COUNT(*)"
            elif agg == 'COUNT_DISTINCT':
                select += f"COUNT(DISTINCT {col}) AS {col}_count_distinct"
            else:
                select += f" {agg}({col}) as {col}_{agg.lower()}"
            
            if curr_iter < last_iter:
                select += ","

            curr_iter += 1
    
    return base_query(table, select=select, where=where, order_by=order_by, limit=limit)

## Group By & Aggregate
def groupby_aggregate_query(table, group_by, col_agg_dict, where=None, having=None, order_by=None, limit=None):
    query = aggregate_query(table, col_agg_dict)
    query = query.replace("SELECT", f"SELECT {group_by},")
    query += f" GROUP BY {group_by}"
    query += f" HAVING {having}" if having else ""
    query += f" ORDER BY {order_by}" if order_by else ""
    query += f" LIMIT {limit}" if limit else ""
    return query

## Short-hands 

### Read tables and schemas

def pg_tables_query(public_filter=True):
    where = "schemaname = 'public'" if public_filter else None
    return base_query('pg_catalog.pg_tables', select='*', where=where)

def table_schema_query(table):
    return base_query('information_schema.columns', select='*', where=f"table_name='{table}'")

### Count rows, missing values and get value counts

def count_rows_query(table):
    return base_query(table, 'COUNT(*)')

def count_nulls_query(table, columns):
    last_iter = len(columns) - 1
    select = ""
    for i, col in enumerate(columns): 
        select += f"SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS {col}_null_values"
        if i < last_iter:
            select += ", "

    return base_query(table, select=select)
    
def value_counts_query(table, column, where=None, having=None, order_by=None, limit=None):
    return groupby_aggregate_query(table, column, {column: ['COUNT']}, 
                                   where=where, having=having, order_by=order_by, limit=limit)

In [4]:
load_dotenv()

True

In [5]:
PROTOCOL = 'postgresql+psycopg2'
USER = os.environ.get('POSTGRES_USER')
PASSWORD = os.environ.get('POSTGRES_PASSWORD')
HOST = 'localhost'
PORT = 5432
DB = os.environ.get('POSTGRES_DB')

read_query = get_read_query(PROTOCOL, USER, PASSWORD, HOST, PORT, DB)

In [6]:
query = pg_tables_query()
read_query(query)

SELECT * FROM pg_catalog.pg_tables WHERE schemaname = 'public' 



,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,distribution_centers,user,None,True,False,True,False
1,public,products,user,None,True,False,True,False
2,public,inventory_items,user,None,True,False,True,False
3,public,users,user,None,True,False,True,False
4,public,orders,user,None,True,False,True,False
5,public,order_items,user,None,True,False,True,False
6,public,events,user,None,True,False,False,False


In [7]:
query = """
-- INVENTORY_ITEMS
-- Select relevant columns from inventory items and fill empty brand names
WITH inventory_items_fill_brand AS (
    SELECT 
        id,
        cost,
        product_category,
        product_department,
        CASE WHEN product_brand IS NULL THEN '<<NO_BRAND>>' ELSE product_brand END,
        product_name,
        product_distribution_center_id
    FROM inventory_items
), 
-- Fill empty product names
inventory_items_fill_product AS (
    SELECT 
        id,
        cost,
        product_category,
        product_department,
        product_brand,
        CASE 
            WHEN product_name IS NULL 
            THEN CONCAT(product_brand, ' - ', product_department, ' - ', product_category) 
            ELSE product_name 
        END,
        product_distribution_center_id
    FROM inventory_items_fill_brand
), 
-- Join distribution center names
inventory_items_preped AS (
    SELECT ii.*, dc.name AS distribution_center
    FROM inventory_items_fill_product as ii
    JOIN distribution_centers as dc
    ON ii.product_distribution_center_id = dc.id
), 

-- USERS
-- Select relevant columns
users_preped AS (
    SELECT
        id,
        age as user_age,
        gender as user_gender,
        country as user_county,
        city as user_city,
        traffic_source as user_traffic_source
    FROM users
),

-- ORDER_ITEMS
-- Join prepared columns to order items
order_items_preped AS (
    SELECT oi.id, oi.order_id, oi.user_id, oi.product_id, oi.inventory_item_id,
        oi.status, oi.created_at, oi.shipped_at, oi.delivered_at, oi.returned_at, oi.sale_price,
        ii.cost, ii.product_category, ii.product_department, ii.product_brand, ii.product_name, ii.distribution_center,
        u.user_age, u.user_gender, u.user_county, u.user_city, u.user_traffic_source
    FROM order_items AS oi
    JOIN inventory_items_preped AS ii
    ON oi.inventory_item_id = ii.id
    JOIN users_preped AS u
    ON oi.user_id = u.id
    LIMIT 1000
)

SELECT * FROM order_items_preped
"""

In [9]:
read_query(query).dtypes


-- INVENTORY_ITEMS
-- Select relevant columns from inventory items and fill empty brand names
WITH inventory_items_fill_brand AS (
    SELECT 
        id,
        cost,
        product_category,
        product_department,
        CASE WHEN product_brand IS NULL THEN '<<NO_BRAND>>' ELSE product_brand END,
        product_name,
        product_distribution_center_id
    FROM inventory_items
), 
-- Fill empty product names
inventory_items_fill_product AS (
    SELECT 
        id,
        cost,
        product_category,
        product_department,
        product_brand,
        CASE 
            WHEN product_name IS NULL 
            THEN CONCAT(product_brand, ' - ', product_department, ' - ', product_category) 
            ELSE product_name 
        END,
        product_distribution_center_id
    FROM inventory_items_fill_brand
), 
-- Join distribution center names
inventory_items_preped AS (
    SELECT ii.*, dc.name AS distribution_center
    FROM inventory_items_fill_product as ii
   

id                              int64
order_id                        int64
user_id                         int64
product_id                      int64
inventory_item_id               int64
status                         object
created_at             datetime64[ns]
shipped_at             datetime64[ns]
delivered_at           datetime64[ns]
returned_at            datetime64[ns]
sale_price                    float64
cost                          float64
product_category               object
product_department             object
product_brand                  object
product_name                   object
distribution_center            object
user_age                        int64
user_gender                    object
user_county                    object
user_city                      object
user_traffic_source            object
dtype: object

# Order Items

In [7]:
query = table_schema_query('order_items')
order_items_schema = read_query(query)
order_items_schema

SELECT * FROM information_schema.columns WHERE table_name='order_items' 



,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,database,public,order_items,id,1,nextval('order_items_id_seq'::regclass),NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,database,public,order_items,order_id,2,None,NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,database,public,order_items,user_id,3,None,NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,database,public,order_items,product_id,4,None,NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,database,public,order_items,inventory_item_id,5,None,YES,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
5,database,public,order_items,sale_price,11,None,YES,numeric,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
6,database,public,order_items,created_at,7,None,NO,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
7,database,public,order_items,shipped_at,8,None,YES,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
8,database,public,order_items,delivered_at,9,None,YES,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
9,database,public,order_items,returned_at,10,None,YES,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [8]:
query = count_rows_query('order_items')
read_query(query)

SELECT COUNT(*) FROM order_items 



,count
0,181759


In [9]:
query = base_query('order_items', limit='10')
read_query(query)

SELECT * FROM order_items LIMIT 10 



,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40,NaT,NaT,None,0.02
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21,2023-03-15 22:57:00,2023-03-18 01:08:00,None,0.02
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30,2023-12-06 01:20:00,2023-12-10 10:04:00,None,0.02
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19,2023-12-24 16:44:00,NaT,None,0.02
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59,2022-06-19 19:29:00,NaT,None,0.02
5,154022,106052,84693,14235,415780,Shipped,2023-09-19 09:13:08,2023-09-16 09:24:00,NaT,None,0.02
6,67253,46269,37023,14159,181497,Complete,2021-10-31 01:26:46,2021-11-01 17:47:00,2021-11-02 13:29:00,None,0.49
7,116116,80002,63821,14159,313368,Complete,2022-04-01 13:25:52,2022-03-31 05:13:00,2022-04-02 22:18:00,None,0.49
8,28239,19512,15553,14159,76146,Shipped,2023-10-29 08:08:40,2023-10-29 16:59:00,NaT,None,0.49
9,69641,47894,38312,14159,187914,Shipped,2024-01-07 23:06:39,2024-01-10 19:32:00,NaT,None,0.49


In [10]:
query = count_nulls_query('order_items', order_items_schema.column_name)
read_query(query).T

SELECT SUM(CASE WHEN id IS NULL THEN 1 ELSE 0 END) AS id_null_values, SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) AS order_id_null_values, SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) AS user_id_null_values, SUM(CASE WHEN product_id IS NULL THEN 1 ELSE 0 END) AS product_id_null_values, SUM(CASE WHEN inventory_item_id IS NULL THEN 1 ELSE 0 END) AS inventory_item_id_null_values, SUM(CASE WHEN sale_price IS NULL THEN 1 ELSE 0 END) AS sale_price_null_values, SUM(CASE WHEN created_at IS NULL THEN 1 ELSE 0 END) AS created_at_null_values, SUM(CASE WHEN shipped_at IS NULL THEN 1 ELSE 0 END) AS shipped_at_null_values, SUM(CASE WHEN delivered_at IS NULL THEN 1 ELSE 0 END) AS delivered_at_null_values, SUM(CASE WHEN returned_at IS NULL THEN 1 ELSE 0 END) AS returned_at_null_values, SUM(CASE WHEN status IS NULL THEN 1 ELSE 0 END) AS status_null_values FROM order_items 



,0
id_null_values,0
order_id_null_values,0
user_id_null_values,0
product_id_null_values,0
inventory_item_id_null_values,0
sale_price_null_values,0
created_at_null_values,0
shipped_at_null_values,63478
delivered_at_null_values,117918
returned_at_null_values,163527


In [11]:
col_agg_dict = {
    'id': ['COUNT'],
    'order_id': ['COUNT_DISTINCT'],
    'user_id': ['COUNT_DISTINCT'],
    'product_id': ['COUNT_DISTINCT'],
    'inventory_item_id': ['COUNT_DISTINCT'],
    'sale_price': ['MIN', 'MAX', 'AVG'],
    'created_at': ['MIN', 'MAX'],
    'shipped_at': ['MIN', 'MAX'],
    'delivered_at': ['MIN', 'MAX'],
    'returned_at': ['MIN', 'MAX'],
    'status': ['COUNT_DISTINCT']
}
query = aggregate_query('order_items', col_agg_dict)
read_query(query).T

SELECT COUNT(*),COUNT(DISTINCT order_id) AS order_id_count_distinct,COUNT(DISTINCT user_id) AS user_id_count_distinct,COUNT(DISTINCT product_id) AS product_id_count_distinct,COUNT(DISTINCT inventory_item_id) AS inventory_item_id_count_distinct, MIN(sale_price) as sale_price_min, MAX(sale_price) as sale_price_max, AVG(sale_price) as sale_price_avg, MIN(created_at) as created_at_min, MAX(created_at) as created_at_max, MIN(shipped_at) as shipped_at_min, MAX(shipped_at) as shipped_at_max, MIN(delivered_at) as delivered_at_min, MAX(delivered_at) as delivered_at_max, MIN(returned_at) as returned_at_min, MAX(returned_at) as returned_at_max,COUNT(DISTINCT status) AS status_count_distinct FROM order_items 



,0
count,181759
order_id_count_distinct,125226
user_id_count_distinct,80044
product_id_count_distinct,29046
inventory_item_id_count_distinct,181759
sale_price_min,0.02
sale_price_max,999.0
sale_price_avg,59.568544
created_at_min,2019-01-06 02:25:41
created_at_max,2024-01-21 18:02:23.533893


In [12]:
query = value_counts_query('order_items', 'status')
read_query(query)

SELECT status, COUNT(*) FROM order_items GROUP BY status 



,status,count
0,Cancelled,27090
1,Complete,45609
2,Processing,36388
3,Returned,18232
4,Shipped,54440


# Users

In [13]:
query = table_schema_query('users')
users_schema = read_query(query)
users_schema

SELECT * FROM information_schema.columns WHERE table_name='users' 



,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,database,public,users,id,1,nextval('users_id_seq'::regclass),NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,database,public,users,age,5,None,YES,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,database,public,users,latitude,12,None,YES,double precision,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,database,public,users,longitude,13,None,YES,double precision,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,database,public,users,created_at,15,None,NO,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
5,database,public,users,gender,6,None,YES,character varying,10.0,40.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES
6,database,public,users,state,7,None,YES,character varying,50.0,200.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES
7,database,public,users,street_address,8,None,YES,character varying,255.0,1020.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES
8,database,public,users,postal_code,9,None,YES,character varying,20.0,80.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES
9,database,public,users,city,10,None,YES,character varying,255.0,1020.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [14]:
query = count_rows_query('users')
read_query(query)

SELECT COUNT(*) FROM users 



,count
0,100000


In [15]:
query = base_query('users', limit=10)
read_query(query)

SELECT * FROM users LIMIT 10 



,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,457,Timothy,Bush,timothybush@example.net,65,M,Acre,87620 Johnson Hills,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2022-07-19 13:51:00.000000
1,6578,Elizabeth,Martinez,elizabethmartinez@example.com,34,F,Acre,1705 Nielsen Land,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2023-11-08 18:49:00.000000
2,36280,Christopher,Mendoza,christophermendoza@example.net,13,M,Acre,125 Turner Isle Apt. 264,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Email,2019-08-24 06:10:00.000000
3,60193,Jimmy,Conner,jimmyconner@example.com,64,M,Acre,0966 Jose Branch Apt. 008,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2020-02-15 11:26:00.000000
4,64231,Natasha,Wilson,natashawilson@example.net,25,F,Acre,20798 Phillip Trail Apt. 392,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2020-03-13 06:45:00.000000
5,72187,Andrea,Bryant,andreabryant@example.org,47,F,Acre,622 Sims Field,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2022-05-27 14:53:00.000000
6,22528,Keith,Barnett,keithbarnett@example.com,64,M,Acre,53908 Amy Fork Apt. 281,69940-000,Sena Madureira,Brasil,-9.857324,-69.437057,Search,2019-03-21 10:58:00.000000
7,34417,Joanne,Mcpherson,joannemcpherson@example.com,28,F,Acre,53907 John Unions Apt. 099,69940-000,Sena Madureira,Brasil,-9.857324,-69.437057,Search,2024-01-12 19:02:31.096664
8,50098,Ian,Burke,ianburke@example.net,52,M,Acre,91902 Andrea Stream Apt. 563,69940-000,Sena Madureira,Brasil,-9.857324,-69.437057,Email,2019-04-12 06:11:00.000000
9,63352,David,Orozco,davidorozco@example.org,18,M,Acre,9576 Dominique Run,69940-000,Sena Madureira,Brasil,-9.857324,-69.437057,Search,2023-10-29 12:33:00.000000


In [16]:
query = count_nulls_query('users', users_schema.column_name)
read_query(query).T

SELECT SUM(CASE WHEN id IS NULL THEN 1 ELSE 0 END) AS id_null_values, SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS age_null_values, SUM(CASE WHEN latitude IS NULL THEN 1 ELSE 0 END) AS latitude_null_values, SUM(CASE WHEN longitude IS NULL THEN 1 ELSE 0 END) AS longitude_null_values, SUM(CASE WHEN created_at IS NULL THEN 1 ELSE 0 END) AS created_at_null_values, SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS gender_null_values, SUM(CASE WHEN state IS NULL THEN 1 ELSE 0 END) AS state_null_values, SUM(CASE WHEN street_address IS NULL THEN 1 ELSE 0 END) AS street_address_null_values, SUM(CASE WHEN postal_code IS NULL THEN 1 ELSE 0 END) AS postal_code_null_values, SUM(CASE WHEN city IS NULL THEN 1 ELSE 0 END) AS city_null_values, SUM(CASE WHEN country IS NULL THEN 1 ELSE 0 END) AS country_null_values, SUM(CASE WHEN first_name IS NULL THEN 1 ELSE 0 END) AS first_name_null_values, SUM(CASE WHEN last_name IS NULL THEN 1 ELSE 0 END) AS last_name_null_values, SUM(CASE WHEN email IS NULL TH

,0
id_null_values,0
age_null_values,0
latitude_null_values,0
longitude_null_values,0
created_at_null_values,0
gender_null_values,0
state_null_values,0
street_address_null_values,0
postal_code_null_values,0
city_null_values,0


In [17]:
col_agg_dict = {
    'id': ['COUNT'],
    'first_name': ['COUNT_DISTINCT'],
    'last_name': ['COUNT_DISTINCT'],
    'email': ['COUNT_DISTINCT'],
    'age': ['MIN', 'MAX', 'AVG', 'STDDEV'],
    'gender': ['COUNT_DISTINCT'],
    'state': ['COUNT_DISTINCT'],
    'postal_code': ['COUNT_DISTINCT'],
    'city': ['COUNT_DISTINCT'],
    'country': ['COUNT_DISTINCT'],
    'traffic_source': ['COUNT_DISTINCT'],
    'created_at': ['MIN', 'MAX']
}
query = aggregate_query('users', col_agg_dict)
read_query(query).T

SELECT COUNT(*),COUNT(DISTINCT first_name) AS first_name_count_distinct,COUNT(DISTINCT last_name) AS last_name_count_distinct,COUNT(DISTINCT email) AS email_count_distinct, MIN(age) as age_min, MAX(age) as age_max, AVG(age) as age_avg, STDDEV(age) as age_stddev,COUNT(DISTINCT gender) AS gender_count_distinct,COUNT(DISTINCT state) AS state_count_distinct,COUNT(DISTINCT postal_code) AS postal_code_count_distinct,COUNT(DISTINCT city) AS city_count_distinct,COUNT(DISTINCT country) AS country_count_distinct,COUNT(DISTINCT traffic_source) AS traffic_source_count_distinct, MIN(created_at) as created_at_min, MAX(created_at) as created_at_max FROM users 



,0
count,100000
first_name_count_distinct,690
last_name_count_distinct,1000
email_count_distinct,84011
age_min,12
age_max,70
age_avg,41.05479
age_stddev,17.043914
gender_count_distinct,2
state_count_distinct,229


In [18]:
# Check duplicate users by email
query = value_counts_query('users', 'email', having='COUNT(*)>1')
duplicate_email_counts = read_query(query)
duplicate_email_counts

SELECT email, COUNT(*) FROM users GROUP BY email HAVING COUNT(*)>1 



,email,count
0,michellehunter@example.net,4
1,williamreynolds@example.org,2
2,stephaniegarcia@example.org,3
3,meganlopez@example.org,2
4,thomasthomas@example.com,2
...,...,...
10480,tylerdavis@example.com,2
10481,karensutton@example.net,2
10482,karenjordan@example.net,2
10483,austinbryant@example.com,2


In [19]:
# Check duplicate users
duplicate_emails = "('" + "', '".join(duplicate_email_counts['email']) + "')"
query = base_query('users', where=f'email IN {duplicate_emails}', order_by='email')
read_query(query, verbose=False)

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,35478,Aaron,Allen,aaronallen@example.net,66,M,Guangdong,3529 Justin Harbor Apt. 137,510000,Fuzhou,China,23.237922,113.275174,Display,2021-06-17 17:55:00
1,84974,Aaron,Allen,aaronallen@example.net,18,M,Guangdong,7634 Jose Gateway,510900,Chengdu,China,23.557367,113.568267,Search,2021-09-11 08:47:00
2,99718,Aaron,Brown,aaronbrown@example.com,38,M,Shanghai,34986 Robin Springs Apt. 800,200090,Taiyuan,China,31.263285,121.548417,Search,2022-11-25 16:05:00
3,54655,Aaron,Brown,aaronbrown@example.com,38,M,Texas,4827 Quinn Ridge Apt. 264,76205,Denton,United States,33.190946,-97.130614,Search,2020-05-27 09:13:00
4,21389,Aaron,Brown,aaronbrown@example.org,40,M,Rio Grande do Sul,9249 Lauren Mall,95670-000,Gramado,Brasil,-29.386288,-50.899687,Organic,2022-10-25 11:27:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26469,7720,Zachary,Taylor,zacharytaylor@example.net,68,M,Hubei,153 Jessica Meadows Suite 443,430064,Jinan,China,30.525542,114.286599,Search,2019-03-22 16:28:00
26470,24968,Zachary,Thomas,zacharythomas@example.org,39,M,Illinois,0720 Heather Cliffs Apt. 254,60025,Glenview,United States,42.074961,-87.820115,Search,2021-03-01 14:19:00
26471,775,Zachary,Thomas,zacharythomas@example.org,38,M,Wales,52315 Andrew Turnpike,LL18,Rhyl,United Kingdom,53.296474,-3.436193,Email,2023-09-15 18:38:00
26472,61660,Zachary,Walker,zacharywalker@example.net,29,M,Sichuan,49639 Hogan Rapid,635200,Liuzhou,China,30.818069,106.945046,Search,2019-01-25 12:23:00


In [20]:
# Check if enum columns are valid
enum_cols = ['gender', 'city', 'state', 'country', 'traffic_source']
for col in enum_cols:
    query = value_counts_query('users', col, order_by='count DESC')
    print(read_query(query), '\n')
    print(50*'-', '\n')

SELECT gender, COUNT(*) FROM users GROUP BY gender ORDER BY count DESC 

  gender  count
0      F  50208
1      M  49792 

-------------------------------------------------- 

SELECT city, COUNT(*) FROM users GROUP BY city ORDER BY count DESC 

             city  count
0        Shanghai   2525
1         Beijing   2175
2           Seoul   1483
3        Shenzhen   1320
4            null    958
...           ...    ...
7879    Sammamish      1
7880       Gavere      1
7881  Erquelinnes      1
7882       Veurne      1
7883        Borgo      1

[7884 rows x 2 columns] 

-------------------------------------------------- 

SELECT state, COUNT(*) FROM users GROUP BY state ORDER BY count DESC 

          state  count
0     Guangdong   5380
1       England   4034
2    California   3704
3      Shanghai   2499
4         Texas   2468
..          ...    ...
224    Nagasaki      2
225     Tochigi      2
226  Vorarlberg      2
227        Nara      1
228      Kagawa      1

[229 rows x 2 columns] 

--

In [21]:
query = value_counts_query('users', 'city', order_by='count DESC')
read_query(query)

SELECT city, COUNT(*) FROM users GROUP BY city ORDER BY count DESC 



,city,count
0,Shanghai,2525
1,Beijing,2175
2,Seoul,1483
3,Shenzhen,1320
4,null,958
...,...,...
7879,Sammamish,1
7880,Gavere,1
7881,Erquelinnes,1
7882,Veurne,1


In [22]:
query = value_counts_query('users', 'city', order_by='city')
read_query(query).city.tolist()

SELECT city, COUNT(*) FROM users GROUP BY city ORDER BY city 



['Aachen',
 'Aalst',
 'Aalter',
 'Abaetetuba',
 'Abbaretz',
 'Abbeville',
 'Abbots Bromley',
 'Aberdare',
 'Aberdeen',
 'Abilene',
 'Abingdon',
 'Abrera',
 'Abrest',
 'Açailândia',
 'Acará',
 'Acaraú',
 'Accrington',
 'Achères',
 'Achern',
 'Achim',
 'Acworth',
 'Ada',
 'Addison',
 'Addlestone',
 'Adelaide',
 'Adelanto',
 'Adelphi',
 'Affligem',
 'Affton',
 'Afogados da Ingazeira',
 'Agadir',
 'Agde',
 'Agen',
 'Agnetz',
 'Agoura Hills',
 'Agrovila Liberal',
 'Aguadulce',
 'Aguaí',
 'Águas Belas',
 'Águas Lindas de Goiás',
 'Agudos',
 'Aguilar de la Frontera',
 'Águilas',
 'Ahaus',
 'Ahlen',
 'Ahrensburg',
 'Aichach',
 'Aigaliers',
 'Aigondigné',
 'Aikawa-cho',
 'Aiken',
 'Airdrie',
 'Aire-sur-la-Lys',
 'Airmont',
 'Aiseau-Presles',
 'Aix-en-Provence',
 'Aix-Noulette',
 'Aizenay',
 'Ajaccio',
 'Akashi-shi',
 'Ak-Chin Indian Community',
 'Akiruno City',
 'Akishima City',
 'Aksu area',
 'Alabaster',
 'Alacuás',
 'Alameda',
 'Alamo',
 'Alamogordo',
 'Alayor',
 'Albacete',
 'Alba-la-Romain

# Inventory Items

In [23]:
query = table_schema_query('inventory_items')
inventory_items_schema = read_query(query)
inventory_items_schema

SELECT * FROM information_schema.columns WHERE table_name='inventory_items' 



,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,database,public,inventory_items,id,1,nextval('inventory_items_id_seq'::regclass),NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,database,public,inventory_items,product_id,2,None,NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,database,public,inventory_items,created_at,3,None,NO,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,database,public,inventory_items,sold_at,4,None,YES,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,database,public,inventory_items,cost,5,None,NO,numeric,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
5,database,public,inventory_items,product_retail_price,9,None,YES,numeric,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
6,database,public,inventory_items,product_distribution_center_id,12,None,NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
7,database,public,inventory_items,product_brand,8,None,YES,character varying,255.0,1020.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES
8,database,public,inventory_items,product_sku,11,None,YES,character varying,255.0,1020.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES
9,database,public,inventory_items,product_department,10,None,YES,character varying,255.0,1020.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [24]:
query = count_rows_query('inventory_items')
read_query(query)

SELECT COUNT(*) FROM inventory_items 



,count
0,490705


In [26]:
query = base_query('inventory_items', limit=10)
read_query(query)

SELECT * FROM inventory_items LIMIT 10 



,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,67971,13844,2022-07-02 07:09:20,2022-07-24 06:33:20,2.77,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
1,67972,13844,2023-12-20 03:28:00,NaT,2.77,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
2,67973,13844,2023-06-04 02:53:00,NaT,2.77,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
3,72863,13844,2021-10-16 22:58:52,2021-11-22 02:19:52,2.77,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
4,72864,13844,2021-08-07 16:33:00,NaT,2.77,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
5,101826,13844,2023-10-22 23:08:09,2023-11-15 13:49:09,2.77,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
6,101827,13844,2023-10-10 08:15:00,NaT,2.77,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
7,107797,13844,2023-12-27 05:50:59,2024-01-13 23:58:59,2.77,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
8,107798,13844,2020-12-06 18:13:00,NaT,2.77,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
9,107799,13844,2020-01-30 04:46:00,NaT,2.77,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7


In [94]:
query = count_nulls_query('inventory_items', inventory_items_schema.column_name)
read_query(query).T

SELECT SUM(CASE WHEN id IS NULL THEN 1 ELSE 0 END) AS id_null_values, SUM(CASE WHEN product_id IS NULL THEN 1 ELSE 0 END) AS product_id_null_values, SUM(CASE WHEN created_at IS NULL THEN 1 ELSE 0 END) AS created_at_null_values, SUM(CASE WHEN sold_at IS NULL THEN 1 ELSE 0 END) AS sold_at_null_values, SUM(CASE WHEN cost IS NULL THEN 1 ELSE 0 END) AS cost_null_values, SUM(CASE WHEN product_retail_price IS NULL THEN 1 ELSE 0 END) AS product_retail_price_null_values, SUM(CASE WHEN product_distribution_center_id IS NULL THEN 1 ELSE 0 END) AS product_distribution_center_id_null_values, SUM(CASE WHEN product_brand IS NULL THEN 1 ELSE 0 END) AS product_brand_null_values, SUM(CASE WHEN product_sku IS NULL THEN 1 ELSE 0 END) AS product_sku_null_values, SUM(CASE WHEN product_department IS NULL THEN 1 ELSE 0 END) AS product_department_null_values, SUM(CASE WHEN product_category IS NULL THEN 1 ELSE 0 END) AS product_category_null_values, SUM(CASE WHEN product_name IS NULL THEN 1 ELSE 0 END) AS produ

,0
id_null_values,0
product_id_null_values,0
created_at_null_values,0
sold_at_null_values,308946
cost_null_values,0
product_retail_price_null_values,0
product_distribution_center_id_null_values,0
product_brand_null_values,401
product_sku_null_values,0
product_department_null_values,0


In [95]:
col_agg_dict = {
    'id': ['COUNT'],
    'product_name': ['COUNT_DISTINCT'],
    'product_category': ['COUNT_DISTINCT'],
    'product_brand': ['COUNT_DISTINCT'],
    'product_department': ['COUNT_DISTINCT'],
    'product_distribution_center_id': ['COUNT_DISTINCT'],
    'cost': ['MIN', 'MAX', 'AVG', 'STDDEV'],
    'product_retail_price': ['MIN', 'MAX', 'AVG', 'STDDEV']
}
query = aggregate_query('inventory_items', col_agg_dict)
read_query(query).T

SELECT COUNT(*),COUNT(DISTINCT product_name) AS product_name_count_distinct,COUNT(DISTINCT product_category) AS product_category_count_distinct,COUNT(DISTINCT product_brand) AS product_brand_count_distinct,COUNT(DISTINCT product_department) AS product_department_count_distinct,COUNT(DISTINCT product_distribution_center_id) AS product_distribution_center_id_count_distinct, MIN(cost) as cost_min, MAX(cost) as cost_max, AVG(cost) as cost_avg, STDDEV(cost) as cost_stddev, MIN(product_retail_price) as product_retail_price_min, MAX(product_retail_price) as product_retail_price_max, AVG(product_retail_price) as product_retail_price_avg, STDDEV(product_retail_price) as product_retail_price_stddev FROM inventory_items 



,0
count,490705.000000
product_name_count_distinct,27236.000000
product_category_count_distinct,26.000000
product_brand_count_distinct,2752.000000
product_department_count_distinct,2.000000
product_distribution_center_id_count_distinct,10.000000
cost_min,0.010000
cost_max,557.150000
cost_avg,28.659112
cost_stddev,31.009454


In [96]:
# Empty product names
query = base_query('inventory_items', where='product_name IS NULL', limit=10)
read_query(query)

SELECT * FROM inventory_items WHERE product_name IS NULL LIMIT 10 



,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,59968,12586,2023-12-06 15:30:31,2023-12-23 03:42:31,18.97,Intimates,None,Josie by Natori,36.0,Women,A7EA034186E14FB5F7B37CF664893CD2,1
1,59969,12586,2020-03-16 08:56:00,NaT,18.97,Intimates,None,Josie by Natori,36.0,Women,A7EA034186E14FB5F7B37CF664893CD2,1
2,82611,12586,2023-04-04 12:29:10,2023-05-18 23:09:10,18.97,Intimates,None,Josie by Natori,36.0,Women,A7EA034186E14FB5F7B37CF664893CD2,1
3,82612,12586,2022-07-03 05:10:00,NaT,18.97,Intimates,None,Josie by Natori,36.0,Women,A7EA034186E14FB5F7B37CF664893CD2,1
4,82613,12586,2020-06-04 06:18:00,NaT,18.97,Intimates,None,Josie by Natori,36.0,Women,A7EA034186E14FB5F7B37CF664893CD2,1
5,82614,12586,2021-07-24 16:16:00,NaT,18.97,Intimates,None,Josie by Natori,36.0,Women,A7EA034186E14FB5F7B37CF664893CD2,1
6,215006,12586,2021-08-26 13:38:45,2021-10-05 02:31:45,18.97,Intimates,None,Josie by Natori,36.0,Women,A7EA034186E14FB5F7B37CF664893CD2,1
7,215007,12586,2020-04-26 02:42:00,NaT,18.97,Intimates,None,Josie by Natori,36.0,Women,A7EA034186E14FB5F7B37CF664893CD2,1
8,215008,12586,2022-05-15 09:06:00,NaT,18.97,Intimates,None,Josie by Natori,36.0,Women,A7EA034186E14FB5F7B37CF664893CD2,1
9,215009,12586,2023-05-14 01:06:00,NaT,18.97,Intimates,None,Josie by Natori,36.0,Women,A7EA034186E14FB5F7B37CF664893CD2,1


In [97]:
# Empty product brands
query = base_query('inventory_items', where='product_brand IS NULL', limit=10)
read_query(query)

SELECT * FROM inventory_items WHERE product_brand IS NULL LIMIT 10 



,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,167148,11668,2023-10-31 02:21:06,2023-11-25 13:27:06,18.66,Intimates,Everyday Comfort Cami,None,38.4,Women,67E4423AFD0FFD2F451890C0888E6F33,3
1,167149,11668,2022-08-09 04:18:00,NaT,18.66,Intimates,Everyday Comfort Cami,None,38.4,Women,67E4423AFD0FFD2F451890C0888E6F33,3
2,167150,11668,2023-02-05 06:07:00,NaT,18.66,Intimates,Everyday Comfort Cami,None,38.4,Women,67E4423AFD0FFD2F451890C0888E6F33,3
3,167151,11668,2022-12-04 10:51:00,NaT,18.66,Intimates,Everyday Comfort Cami,None,38.4,Women,67E4423AFD0FFD2F451890C0888E6F33,3
4,320906,11668,2022-11-14 20:27:55,2022-12-14 07:21:55,18.66,Intimates,Everyday Comfort Cami,None,38.4,Women,67E4423AFD0FFD2F451890C0888E6F33,3
5,320907,11668,2021-11-16 18:41:00,NaT,18.66,Intimates,Everyday Comfort Cami,None,38.4,Women,67E4423AFD0FFD2F451890C0888E6F33,3
6,320908,11668,2020-03-14 07:28:00,NaT,18.66,Intimates,Everyday Comfort Cami,None,38.4,Women,67E4423AFD0FFD2F451890C0888E6F33,3
7,320909,11668,2023-04-27 00:50:00,NaT,18.66,Intimates,Everyday Comfort Cami,None,38.4,Women,67E4423AFD0FFD2F451890C0888E6F33,3
8,334110,11668,2023-01-11 10:31:48,2023-01-29 10:55:48,18.66,Intimates,Everyday Comfort Cami,None,38.4,Women,67E4423AFD0FFD2F451890C0888E6F33,3
9,334111,11668,2020-09-21 09:15:00,NaT,18.66,Intimates,Everyday Comfort Cami,None,38.4,Women,67E4423AFD0FFD2F451890C0888E6F33,3


In [98]:
categorical_columns = ['product_name', 'product_category', 'product_brand', 'product_department', 'product_distribution_center_id']
for col in categorical_columns:
    query = value_counts_query('inventory_items', col, order_by='COUNT(*) DESC')
    print(read_query(query), '')
    print(50*'-', '\n')

SELECT product_name, COUNT(*) FROM inventory_items GROUP BY product_name ORDER BY COUNT(*) DESC 

                                            product_name  count
0      Wrangler Men's Premium Performance Cowboy Cut ...    161
1                                       Puma Men's Socks    130
2      7 For All Mankind Men's Standard Classic Strai...    117
3                True Religion Men's Ricky Straight Jean    105
4                   Kenneth Cole Men's Straight Leg Jean    103
...                                                  ...    ...
27232     Tommy Hilfiger Men Fashion V-Neck Sweater VEST      2
27233  Devon & Jones D730 Mens Three-Season Sport Jacket      2
27234                   Anne Klein Women's Classic Skirt      2
27235  Lucky Brand Jeans Women's Rose Print Full Zip ...      2
27236  Bloom's Outlet Sexy Black Double Layer Crochet...      2

[27237 rows x 2 columns] 
-------------------------------------------------- 

SELECT product_category, COUNT(*) FROM inventory_items

# Distribution Centers

In [100]:
query = table_schema_query('distribution_centers')
inventory_items_schema = read_query(query)
inventory_items_schema

SELECT * FROM information_schema.columns WHERE table_name='distribution_centers' 



,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,database,public,distribution_centers,id,1,nextval('distribution_centers_id_seq'::regclass),NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,database,public,distribution_centers,latitude,3,None,NO,double precision,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,database,public,distribution_centers,longitude,4,None,NO,double precision,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,database,public,distribution_centers,name,2,None,NO,character varying,255.0,1020.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [101]:
query = count_rows_query('distribution_centers')
read_query(query)

SELECT COUNT(*) FROM distribution_centers 



,count
0,10


In [102]:
query = base_query('distribution_centers')
read_query(query)

SELECT * FROM distribution_centers 



,id,name,latitude,longitude
0,1,Memphis TN,35.1174,-89.9711
1,2,Chicago IL,41.8369,-87.6847
2,3,Houston TX,29.7604,-95.3698
3,4,Los Angeles CA,34.0500,-118.2500
4,5,New Orleans LA,29.9500,-90.0667
5,6,Port Authority of New York/New Jersey NY/NJ,40.6340,-73.7834
6,7,Philadelphia PA,39.9500,-75.1667
7,8,Mobile AL,30.6944,-88.0431
8,9,Charleston SC,32.7833,-79.9333
9,10,Savannah GA,32.0167,-81.1167


# Notes

**Order Items:**
- Exclude cancelled and returned order items from the rest of the analysis. Show them as a metric.

**Inventory Items:**
- Fill missing brands with "(NONAME)".
- Fill missing product names with "{brand} - {department} - {category}".

**Users:**
- In case of duplicate emails: Get the last values from users table, merge the history.
- Standardize country names. (España -> Spain, Deutschland -> Germany)